In [1]:
# imports
import torch
from torch.utils.data import Dataset, DataLoader, random_split, SequentialSampler, RandomSampler, BatchSampler
from torchvision import models, transforms
from torch import nn, optim

from PIL import Image
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from utils.clean_labels import clean_labels
from utils.prepare_images import prepare_images
from utils.mydataset import SVRCDataset

import os
from datetime import datetime
import random
import time

%load_ext autoreload
%autoreload 2
print(torch.version.cuda)

11.3


In [2]:
# put videos here!
# video_base = 'data/videos'
video_base = 'D:/e6691/6691_assignment2/videos'
videos = [vid for vid in os.listdir(video_base) if vid.endswith('mp4')]
# images will be output to here
# image_base = 'data/images'
image_base = 'D:/e6691/6691_assignment2/images'
if not os.path.exists(image_base):
    os.mkdir(image_base)

In [ ]:
#!rm -r {image_base + '/*'}

In [3]:
videos

['RALIHR_surgeon01_fps01_0001.mp4',
 'RALIHR_surgeon01_fps01_0002.mp4',
 'RALIHR_surgeon01_fps01_0003.mp4',
 'RALIHR_surgeon01_fps01_0004.mp4',
 'RALIHR_surgeon01_fps01_0005.mp4',
 'RALIHR_surgeon01_fps01_0006.mp4',
 'RALIHR_surgeon01_fps01_0007.mp4',
 'RALIHR_surgeon01_fps01_0008.mp4',
 'RALIHR_surgeon01_fps01_0009.mp4',
 'RALIHR_surgeon01_fps01_0010.mp4',
 'RALIHR_surgeon01_fps01_0011.mp4',
 'RALIHR_surgeon01_fps01_0012.mp4',
 'RALIHR_surgeon01_fps01_0013.mp4',
 'RALIHR_surgeon01_fps01_0014.mp4',
 'RALIHR_surgeon01_fps01_0015.mp4',
 'RALIHR_surgeon01_fps01_0016.mp4',
 'RALIHR_surgeon01_fps01_0017.mp4',
 'RALIHR_surgeon01_fps01_0018.mp4',
 'RALIHR_surgeon01_fps01_0019.mp4',
 'RALIHR_surgeon01_fps01_0020.mp4',
 'RALIHR_surgeon01_fps01_0021.mp4',
 'RALIHR_surgeon01_fps01_0022.mp4',
 'RALIHR_surgeon01_fps01_0023.mp4',
 'RALIHR_surgeon01_fps01_0024.mp4',
 'RALIHR_surgeon01_fps01_0025.mp4',
 'RALIHR_surgeon01_fps01_0026.mp4',
 'RALIHR_surgeon01_fps01_0027.mp4',
 'RALIHR_surgeon01_fps01_002

In [10]:
# command line: 
# ffmpeg -i {input_video} -r {frame_rate} [-f {force_format} (not needed)] {output_images}
# doc: https://ffmpeg.org/ffmpeg.html
for video in videos:
    # input_path = os.path.join(video_base, video)
    input_path = video_base + '/' + video
    # make dirs
    output_base = image_base + '/{}'.format(video.split('.')[0])
    if not os.path.exists(output_base):
        os.mkdir(output_base)
    #output_path = os.path.join(output_base, '%d.png')
    output_path = output_base + '/' + '%d.png'
    # # command
    # print('Frames extracted from {} to {}'.format(input_path, output_path))
    # !ffmpeg -i {input_path} -r 1 {output_path}

In [11]:
labels_path = 'data/labels/video.phase.trainingData.clean.StudentVersion.csv'
names_path = 'data/labels/all_labels_hernia.csv'
labels_df = pd.read_csv(labels_path)
names_df = pd.read_csv(names_path, index_col='id')

In [12]:
names_df

,labels
id,
0,adhesiolysis
1,blurry
2,catheter insertion
3,mesh placement
4,mesh positioning
5,out of body
6,peritoneal closure
7,peritoneal scoring
8,positioning suture


In [13]:
prepare_images(video_base, image_base, labels_df, names_df, 'png')

d:\python\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
